In [86]:
import supriya
from supriya import Envelope, synthdef
from supriya.ugens import EnvGen, Out, SinOsc

In [87]:
server = supriya.Server().connect(ip_address="172.28.160.1", port=57110)

In [154]:
def ignore(*whatevs): pass
#server.osc_protocol.osc_server.handle_error = ignore.__get__(server.osc_protocol.osc_server)
supriya.osc.threaded.ThreadedOscProtocol.Server.handle_error = ignore

In [ ]:
server._add_requests(supriya.contexts.requests.NewGroup(items=[(1,0,0)]))

haha


In [99]:
print(server.query_tree())

NODE TREE 0 group
    1 group
        1000 group



In [90]:
group = server.default_group.add_group()

In [91]:
group

/home/jleen/src/collide/.venv/lib/python3.14/site-packages/supriya/contexts/realtime.py:236: FailWarning: /g_new Group 2 not found
  warnings.warn(


Group(context=<Server ONLINE [/usr/local/bin/scsynth -B 172.28.160.1 -R 0 -l 1 -u 57110]>, id_=67109864, parallel=False)

In [92]:
@synthdef()
def simple_sine(frequency=440, amplitude=0.1, gate=1):
    sine = SinOsc.ar(frequency=frequency) * amplitude
    envelope = EnvGen.kr(envelope=Envelope.adsr(), gate=gate, done_action=2)
    Out.ar(bus=0, source=[sine * envelope] * 2)

In [93]:
server.add_synthdefs(simple_sine)
server.sync()

<Server ONLINE [/usr/local/bin/scsynth -B 172.28.160.1 -R 0 -l 1 -u 57110]>

In [94]:
foo = group.add_synth(simple_sine, frequency=770/2)


In [95]:
group.free_children()

/home/jleen/src/collide/.venv/lib/python3.14/site-packages/supriya/contexts/realtime.py:236: FailWarning: /s_new Group 67109864 not found
  warnings.warn(


In [96]:
# touch /usr/local/bin/scsynth
foo.__repr__()

/home/jleen/src/collide/.venv/lib/python3.14/site-packages/supriya/contexts/realtime.py:236: FailWarning: /g_freeAll Group 67109864 not found
  warnings.warn(


'Synth(context=<Server ONLINE [/usr/local/bin/scsynth -B 172.28.160.1 -R 0 -l 1 -u 57110]>, id_=67109865, synthdef=<SynthDef: simple_sine>)'

In [97]:
# apt install graphviz
supriya.graph(simple_sine)

(PosixPath('/home/jleen/.cache/supriya/2025-11-07T20-43-06-219324-e53ac9a.pdf'),
 0.0023963451385498047,
 0.019796371459960938,
 True,
 'dot - graphviz version 2.43.0 (0)\nUsing render: cairo:cairo\nUsing device: pdf:cairo:cairo\nlibdir = "/usr/lib/x86_64-linux-gnu/graphviz"\nActivated plugin library: libgvplugin_dot_layout.so.6\nUsing layout: dot:dot_layout\nThe plugin configuration file:\n\t/usr/lib/x86_64-linux-gnu/graphviz/config6a\n\t\twas successfully loaded.\n    render\t:  cairo dot dot_json fig gd json json0 map mp pic pov ps svg tk visio vml vrml xdot xdot_json\n    layout\t:  circo dot fdp neato nop nop1 nop2 osage patchwork sfdp twopi\n    textlayout\t:  textlayout\n    device\t:  canon cmap cmapx cmapx_np dot dot_json eps fig gd gd2 gif gv imap imap_np ismap jpe jpeg jpg json json0 mp pdf pic plain plain-ext png pov ps ps2 svg svgz tk vdx vml vmlz vrml wbmp webp x11 xdot xdot1.2 xdot1.4 xdot_json xlib\n    loadimage\t:  (lib) eps gd gd2 gif jpe jpeg jpg png ps svg webp xbm

In [98]:
import graphviz